In [10]:
#import spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

import credentials

In [12]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=credientals.SPOTIPY_CLIENT_ID,
                                               client_secret=credientals.SPOTIPY_CLIENT_SECRET,
                                               redirect_uri=credientals.SPOTIPY_REDIRECT_URI,
                                               scope="user-library-read"))


### General Understanding
Gather data from a specific spotify playlist and see what we can do with it

In [3]:
# Warm & Fuzzy Playlist from Spotify
playlist_data = sp.user_playlist_tracks("spotify", "37i9dQZF1DX5IDTimEWoTd")
#print(playlist_data.keys())
num_tracks = len(playlist_data['items'])
tracks = []
genres = []

#collect tracks ID 
for i in range (num_tracks):
    #toxic_id = toxic_id.replace("spotify:track:", "")
    song_id = playlist_data['items'][i]['track']['uri']
    song_id = song_id.replace("spotify:track:", "")
    #artist_info = playlist_data['artist_uri']
    tracks.append(song_id)

print(tracks)

['2769RhOZs0ezRncRW1vryp', '1Y3LN4zO1Edc2EluIoSPJN', '2IOFZdYYkFxEHVz1w34PoL', '1XNAH3h63O1hWFcgyHGbjf', '7Lv78wiIJcKgND9Zi7AIOt', '4A2LfnduSTsE8u0ecYROxE', '10UTmRvoroe6eZ3ViwkpbN', '50x1Ic8CaXkYNvjmxe3WXy', '6I0KEvFfWszVJiCy5hum4V', '1aEsTgCsv8nOjEgyEoRCpS', '6VsvKPJ4xjVNKpI8VVZ3SV', '4ZHGRFGwjnEIXHG2FhcDPv', '2PR4jVXzyjCky2PkONvipq', '1Mk9ou2m6hsEZJYuNRJFpO', '79qxwHypONUt3AFq0WPpT9', '0OPIBKjAYBFzNpo1HzjqzQ', '6vC7mF2CXYW6MVF2voO4FU', '09fAL7YwPV3YzVmQDzLY8d', '5odlY52u43F5BjByhxg7wg', '07koEqsKHZTlGVMC9eoEjO', '0wavGRldH0AWyu2zvTz8zb', '0hiHNxqKGf3GZgVTIOAiiK', '49FwhOuCZ9LWaEpGNal2lQ', '0YMe6PHRbeDcN7KJdCG0bW', '2XkDm5m2vPowecEAAR5gmb', '093KMwH96ZwZRZn2lKoX9Z', '2Zy7SjGT12yzvFohfAKWDq', '7b9oZ5tuGDKRWsL0lgQXqg', '4XxUka4jIa0girDclOY4iA', '5iMUQTIxTGgbVItGIPAWbL', '1dGr1c8CrMLDpV6mPbImSI', '6IPwKM3fUUzlElbvKw2sKl', '6qsacYwaDcrOy4CnTDxWNF', '1hCTxutVVYvggAQcUUUZAT', '6FtIK0IkmG33lKbc1pyAlZ', '0lx2cLdOt3piJbcaXIV74f', '3JjnGLK8IxkNLvo8Lb3KOM', '4sFhD0PjpNgV8M8yIzt7C2', '1is8gU4RVc

## Create a function that will turn the playlist data into a dataframe

In [4]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [5]:
analyzed_playlist_data = analyze_playlist("spotify", "37i9dQZF1DX5IDTimEWoTd")
analyzed_playlist_data.head()

/Users/victoria/Documents/me/MusicDataAnalysis/.env/lib/python3.5/site-packages/ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,album,artist,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_id,track_name,valence
0,Till Forever Falls Apart,Ashe,0.509,222861,0.498,0,2,0.0674,-6.940,1,0.0315,99.729,4,2769RhOZs0ezRncRW1vryp,Till Forever Falls Apart,0.283
1,Until I Found You (Em Beihold Version),Stephen Sanchez,0.551,176440,0.550,0,10,0.2020,-5.339,1,0.0286,101.190,3,1Y3LN4zO1Edc2EluIoSPJN,Until I Found You (with Em Beihold) - Em Beiho...,0.342
2,Fine Line,Harry Styles,0.530,259187,0.512,0.00453,7,0.1290,-9.261,1,0.0259,95.946,4,2IOFZdYYkFxEHVz1w34PoL,Cherry,0.411
3,LEGEND,John Legend,0.545,188046,0.587,0,2,0.2240,-4.842,1,0.0634,123.107,4,1XNAH3h63O1hWFcgyHGbjf,Nervous,0.359
4,Fall Into Me,Forest Blakk,0.450,226067,0.483,0,2,0.1270,-6.754,1,0.0572,118.305,4,7Lv78wiIJcKgND9Zi7AIOt,Fall Into Me,0.206


In [6]:
analyzed_playlist_data['danceability'].mean()

0.53335

In [7]:
# Compute the average values of each feature
averages = analyzed_playlist_data.mean()

# Calculate the variance of each feature
variances = analyzed_playlist_data.var()

print(averages)

danceability      0.533350
energy            0.376598
liveness          0.140060
loudness         -9.751330
speechiness       0.047063
tempo           115.034120
valence           0.355228
dtype: float64


## Access Genere Data

In [8]:
# getting genres
x = playlist_data["items"][0]["track"]['artists']
artist_uri = x[0]["uri"]
artist_info = sp.artist(artist_uri)

artist_genres = artist_info["genres"]

print(artist_genres)

['alt z', 'electropop', 'pop']


In [9]:
# popularity
track_pop = track["track"]["popularity"]

NameError: name 'track' is not defined